In [1]:
import os
import numpy as np
import torchvision
import matplotlib.pyplot as plt
import torch.utils.data as data
import json
import torchvision.transforms.v2 as tfs
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import kaggle
import pandas as pd
import shutil
import time

from tensorflow.keras.datasets import mnist
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torchvision.datasets import ImageFolder
from torchvision import models
from torchvision.models import resnet50, ResNet50_Weights
from torch.utils.data import DataLoader

In [2]:
1

1

In [54]:
train_csv = train_csv.applymap(lambda x: x.replace("train_data/", "") if isinstance(x, str) else x)

C:\Users\ivan2\AppData\Local\Temp\ipykernel_18396\3146484978.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  train_csv = train_csv.applymap(lambda x: x.replace("train_data/", "") if isinstance(x, str) else x)


In [60]:
train_csv.to_csv("dataset/train.csv", index=False)

In [3]:
df = pd.read_csv('dataset/train.csv')
df = df[["file_name", "label"]] 
print(df.head())
print(df.columns)

                              file_name  label
0  a6dcb93f596a43249135678dfcfc17ea.jpg      1
1  041be3153810433ab146bc97d5af505c.jpg      0
2  615df26ce9494e5db2f70e57ce7a3a4f.jpg      1
3  8542fe161d9147be8e835e50c0de39cd.jpg      0
4  5d81fa12bc3b4cea8c94a6700a477cf2.jpg      1
Index(['file_name', 'label'], dtype='object')


In [5]:
csv_file = "dataset/train.csv"
root_dir = "dataset"

In [75]:
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)

In [83]:
train_df.head()

,file_name,label
2530,151ffbe4618645c587112e218a1f6587.jpg,1
11128,5922e1ec46224e67aba3500f6f076e7b.jpg,1
70752,fa1c148c50904732a813053c8512d4dc.jpg,1
44211,134e3edd55ca49669d9fff82f05f9a53.jpg,0
75666,95fee4af52224a8d8468b12f3fbbb343.jpg,1


In [76]:
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [77]:
print(f"Тренировочных данных: {len(train_df)}")
print(f"Валидационных данных: {len(val_df)}")
print(f"Тестовых данных: {len(test_df)}")

Тренировочных данных: 63960
Валидационных данных: 7995
Тестовых данных: 7995


In [95]:
def data_distribution(df: pd.DataFrame, source_path: str, target_path: str):
    for file_name, target in df[['file_name', 'label']].values:
        shutil.move(os.path.join(source_path, file_name), os.path.join(target_path,f"class_{target}", file_name))
    print(f"Данные перемещены в {target_path}")
    

In [96]:
data_distribution(train_df, 'dataset/all_data/', 'dataset/train_data/')

Данные перемещены в dataset/train_data/


In [97]:
data_distribution(val_df, 'dataset/all_data/', 'dataset/val_data/')

Данные перемещены в dataset/val_data/


In [98]:
data_distribution(test_df, 'dataset/all_data/', 'dataset/test_data/')

Данные перемещены в dataset/test_data/


In [5]:
train_transforms = tfs.Compose([
    tfs.RandomResizedCrop(224), 
    tfs.RandomHorizontalFlip(p=0.5),
    tfs.RandomRotation(15),
    tfs.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    tfs.ToTensor(),
    tfs.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Нормализация (значения для ImageNet)
])

C:\Users\ivan2\anaconda3\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [3]:
transforms = tfs.Compose([
    tfs.Resize((224, 224)),        
    tfs.ToTensor(),
    tfs.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Нормализация
])

C:\Users\ivan2\anaconda3\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [4]:
class CustomImageDataset(data.Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.samples = []
        
        for folder_name in os.listdir(root_dir):
            folder_path = os.path.join(root_dir, folder_name)
            
            if not os.path.isdir(folder_path):
                continue
            
            if folder_name == "class_0":
                label = 0
            elif folder_name == "class_1":
                label = 1
            else:
                continue
            
            for file_name in os.listdir(folder_path):
                if file_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                    img_path = os.path.join(folder_path, file_name)
                    self.samples.append((img_path, label))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_path, label = self.samples[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        
        return image, label


In [5]:
train_data_dir = "dataset/train_data"
val_data_dir = "dataset/val_data"
test_data_dir = "dataset/test_data"

In [6]:
train_dataset = CustomImageDataset(root_dir=train_data_dir, transform=transforms)

In [7]:
len(train_dataset)

63960

In [8]:
val_dataset = CustomImageDataset(root_dir=val_data_dir, transform=transforms)
len(val_dataset)

7995

In [9]:
test_dataset = CustomImageDataset(root_dir=test_data_dir, transform=transforms)
len(test_dataset)

7995

In [18]:
model = resnet50(weights=ResNet50_Weights.DEFAULT)

In [19]:
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)

In [30]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [31]:
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [32]:
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [33]:
class Model:
    def __init__(self, model: nn.Module, optimizer: optim,
                train_loader: data.DataLoader, val_loader: data.DataLoader,
                loss_function: nn.Module = nn.CrossEntropyLoss()):
        self.model = model
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        self.loss_function = loss_function

    
    def learn_NN(self, num_epochs: int, patience: int):
        best_model_state = None
        best_val_loss = float('inf')
        patience_counter = 0
        
        start_time = time.time()
        
        for epoch in range(num_epochs):
            if torch.cuda.is_available():
                print(f"Память перед итерацией: {torch.cuda.memory_allocated() / 1e6} MB")
            torch.cuda.empty_cache()
            self.model.train()
            train_loss = 0
            
            train_tqdm = tqdm(self.train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] - Train", leave=False)
            for x_train, y_train in train_tqdm:
                x_train, y_train = x_train.to(self.device), y_train.to(self.device)
                
                self.optimizer.zero_grad()
                predict = self.model(x_train)
                loss = self.loss_function(predict, y_train)
                loss.backward()
                self.optimizer.step()
    
                train_loss += loss.item()
                train_tqdm.set_postfix(loss=f"{loss.item():.4f}")
            
            train_loss /= len(self.train_loader)
            
            self.model.eval()
            val_loss = 0.0
            val_tqdm = tqdm(self.val_loader, desc=f"Epoch [{epoch+1}/{num_epochs}] - Val  ", leave=False)
    
            with torch.no_grad():
                for x_val, y_val in val_tqdm:
                    x_val, y_val = x_val.to(self.device), y_val.to(self.device)
                    predict = self.model(x_val)
                    loss = self.loss_function(predict, y_val)
                    val_loss += loss.item()
                    val_tqdm.set_postfix(loss=f"{loss.item():.4f}")  # Вывод текущей функции потерь
            
            val_loss /= len(self.val_loader)
            
            print(f"Epoch [{epoch+1}/{num_epochs}] - Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
            
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model_state = self.model.state_dict()
                torch.save(best_model_state, f"model_resnet50_best.tar")
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= patience:
                    print(f"Early stopping на эпохе {epoch+1}. Лучшая вал. ошибка: {best_val_loss:.4f}")
                    break    

        if best_model_state is not None:
            self.model.load_state_dict(best_model_state)
            torch.save(best_model_state, f"model_resnet50_best.tar")
            print("Лучшая модель загружена.")
        
        total_time = (time.time() - start_time) / 60  
        print(f"Общее время обучения: {total_time:.2f} минут")

    
    def test_NN(self, test_loader: data.DataLoader):
        self.model.eval()
        test_loss = 0
        correct = 0
        total = 0
        for x_test, y_test in test_loader:
            with torch.no_grad():
                x_test, y_test = x_test.to(self.device), y_test.to(self.device)
                outputs = self.model(x_test)
                predictions = torch.argmax(outputs, dim=1)
                correct += (predictions == y_test).sum().item()
                total += y_test.size(0)
        accuracy = correct / total
        print(f"Test Accuracy: {accuracy:.4f}")

    
    def __str__(self):
        return self.model

In [23]:
checkpoint = torch.load('model_resnet50_1_epoch.tar', weights_only=True)

In [26]:
model.load_state_dict(checkpoint['model'])

<All keys matched successfully>

In [34]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [35]:
resnet1 = Model(model, optimizer, train_loader, val_loader)

In [137]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

In [29]:
torch.cuda.empty_cache()

In [36]:
resnet1.learn_NN(num_epochs=20, patience=5)

Память перед итерацией: 959.327232 MB


Epoch [1/20] - Train Loss: 0.0201 | Val Loss: 0.0162
Память перед итерацией: 1281.206784 MB


Epoch [2/20] - Train Loss: 0.0115 | Val Loss: 0.0092
Память перед итерацией: 1280.121344 MB


Epoch [3/20] - Train Loss: 0.0094 | Val Loss: 0.0079
Память перед итерацией: 1279.20384 MB


Epoch [4/20] - Train Loss: 0.0081 | Val Loss: 0.0070
Память перед итерацией: 1279.924736 MB


...

Epoch [9/20] - Train Loss: 0.0036 | Val Loss: 0.0053
Память перед итерацией: 1280.84224 MB


Epoch [10/20] - Train Loss: 0.0040 | Val Loss: 0.0276
Память перед итерацией: 1281.956352 MB


Epoch [11/20] - Train Loss: 0.0036 | Val Loss: 0.0055
Память перед итерацией: 1280.84224 MB


Epoch [12/20] - Train Loss: 0.0033 | Val Loss: 0.0326
Память перед итерацией: 1281.956352 MB


Epoch [13/20] - Train Loss: 0.0030 | Val Loss: 0.0048
Early stopping на эпохе 13. Лучшая вал. ошибка: 0.0042
Лучшая модель загружена.
Общее время обучения: 508.25 минут


In [37]:
resnet1.test_NN(test_loader)

Test Accuracy: 0.9984


In [89]:
df_control = pd.read_csv('dataset/test_1.csv')
df_control = df_control.map(lambda x: x.replace("test_data_v2/", "") if isinstance(x, str) else x)

In [90]:
df_control['label'] = None

In [91]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [93]:
def control_test(df: pd.DataFrame):
    labels = []
    
    for idx, row in df.iterrows():
        # Получаем путь к изображению
        name = row['id']
        image_path = f"dataset/control_data/{name}"
        
        # Загружаем изображение
        image = Image.open(image_path).convert("RGB")
        
        # Применяем преобразования
        image = transforms(image).unsqueeze(0).to(device)
        
        # Прогоняем изображение через модель
        with torch.no_grad():
            output = resnet1.model(image)
            prediction = torch.argmax(output, dim=1).item()
        
        # Сохраняем предсказание
        labels.append(prediction)

    
    # Добавляем столбец с метками в датафрейм
    df['label'] = labels
    return df

In [94]:
df_control = control_test(df_control)

In [73]:
df_control1 = df_control

In [101]:
df_control1 = df_control1.map(lambda x: x.replace("test_data/", "") if isinstance(x, str) else x)

In [103]:
df_control1['id'] = 'test_data_v2/' + df_control1['id']

In [104]:
df_control1

,id,label
0,test_data_v2/1a2d9fd3e21b4266aea1f66b30aed157.jpg,0
1,test_data_v2/ab5df8f441fe4fbf9dc9c6baae699dc7.jpg,0
2,test_data_v2/eb364dd2dfe34feda0e52466b7ce7956.jpg,0
3,test_data_v2/f76c2580e9644d85a741a42c6f6b39c0.jpg,0
4,test_data_v2/a16495c578b7494683805484ca27cf9f.jpg,0
...,...,...
5535,test_data_v2/483412064ff74d9d9472d606b65976d9.jpg,0
5536,test_data_v2/c0b49ba4081a4197b422dac7c15aea7f.jpg,0
5537,test_data_v2/01454aaedec140c0a3ca1f48028c41cf.jpg,0
5538,test_data_v2/e9adfea8b67e4791968c4c2bdd8ec343.jpg,0


In [105]:
df_control1.to_csv("dataset/test.csv", index=False)